In [1]:
import json
import boto3
import base64
import re
import os
from PIL import Image
import glob
import time
import prompts as p
from importlib import reload

reload(p)
import pandas as pd

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# from langchain_aws import ChatBedrock
bedrock_runtime = boto3.client("bedrock-runtime")

In [2]:
def print_output(output):
    s = pd.Series(output["metadata"])
    display(pd.DataFrame({"Metadata Item": s.index, "Output from AI Model": s.values}))
    bias_list = output["bias_analysis"]
    # Convert to DataFrame with multi-index
    multi_index_data = []
    for i, bias_dict in enumerate(bias_list):
        for key, value in bias_dict.items():
            multi_index_data.append(((i + 1, key), value))

    # Create DataFrame
    multi_index = pd.MultiIndex.from_tuples([item[0] for item in multi_index_data], names=["Bias ID", "Bias Item"])
    df = pd.DataFrame(
        {"Output from AI Model": [item[1] for item in multi_index_data]},
        index=multi_index,
    )
    display(df)

In [3]:
# def convert_png_to_jpg(folder_path):
#     for file_name in os.listdir(folder_path):
#         if file_name.endswith('.png'):
#             png_file_path = os.path.join(folder_path, file_name)
#             jpg_file_name = file_name[:-4] + '.jpg'
#             jpg_file_path = os.path.join(folder_path, jpg_file_name)

#             with Image.open(png_file_path) as img:
#                 rgb_img = img.convert('RGB')
#                 rgb_img.save(jpg_file_path, 'JPEG')

#     print("Conversion completed.")
# convert_png_to_jpg('celebrity_data')

In [4]:
def extract_json_from_text(text):
    # remove chain of thought
    text = text.split("</inclusivity_analysis>")[1].strip()
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        print("Could not decode")
        print(text)
        raise json.JSONDecodeError
    return text


llm_blob = """ Here is the result!!
</inclusivity_analysis>
{"list_items": ["l1", "l2"],
    "nested": {
        "a":1,
        "b":[2]
        }
    }
  """
res = extract_json_from_text(llm_blob)
assert type(res) == dict and res == {
    "list_items": ["l1", "l2"],
    "nested": {"a": 1, "b": [2]},
}

In [5]:
def extract_metadata_from_image(image_path, return_all=False):
    # Read and encode the image
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    # get system and user prompts
    system_prompt = p.system_prompt

    # Create the message with text and image
    messages_dict = {}
    messages_dict["anthropic.claude-3-5-sonnet-20240620-v1:0"] = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": image_data,
                    },
                },
                {"type": "text", "text": p.user_prompt},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": "<inclusivity_analysis>"}],
        },
    ]
    messages_dict["amazon.nova-pro-v1:0"] = [
        {
            "role": "user",
            "content": [
                {
                    # "type": "image",
                    "image": {"format": "jpeg", "source": {"bytes": image_data}}
                },
                {
                    # "type": "text",
                    "text": p.user_prompt
                },
            ],
        },
        {"role": "assistant", "content": [{"text": "<inclusivity_analysis>"}]},
    ]

    request_bodies = {}
    # Prepare the request bodies
    request_bodies["anthropic.claude-3-5-sonnet-20240620-v1:0"] = {
        "anthropic_version": "bedrock-2023-05-31",
        "system": p.system_prompt,
        "max_tokens": 30000,
        "temperature": 0.0,
        "top_p": 0.6,
        # "top_k": 250,
        # "stop_sequences": [''],
        "messages": messages_dict["anthropic.claude-3-5-sonnet-20240620-v1:0"],
    }

    request_bodies["amazon.nova-pro-v1:0"] = {
        "schemaVersion": "messages-v1",
        "messages": messages_dict["amazon.nova-pro-v1:0"],
        "system": [{"text": p.system_prompt}],
        "toolConfig": {},
        "inferenceConfig": {
            "max_new_tokens": 30000,
            "top_p": 0.6,
            "top_k": 250,
            "temperature": 0.0,
            # ,"stopSequences": ['']
        },
    }

    # Send the request to Bedrock
    modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    # modelId='amazon.nova-pro-v1:0'
    # print(modelId)
    # print(request_bodies[modelId])
    response = bedrock_runtime.invoke_model(modelId=modelId, body=json.dumps(request_bodies[modelId]))

    # Process the response
    result = json.loads(response["body"].read())
    if modelId == "anthropic.claude-3-5-sonnet-20240620-v1:0":
        llm_output = result["content"][0]["text"]
    elif modelId == "amazon.nova-pro-v1:0":
        llm_output = result["output"]["message"]["content"][0]["text"]
    else:
        raise RuntimeException("ModelId " + modelId + " must be set up")
    return llm_output if return_all else extract_json_from_text(llm_output)


return_all = True
results = extract_metadata_from_image("celebrity_data/offensive_1.jpg", return_all=return_all)

Be less specific on objects that don't matter, like house, not windows, not parts of the poster, but that it's a poster

make sure to characterize the object itself like that it's a black and white photo

In [6]:
if return_all:
    print("<inclusivity_analysis>")
    print(results)
else:
    print_output(results)

<inclusivity_analysis>


1. Breaking down the image for diversity and inclusion elements:
- Two portraits of men in oval frames
- One man appears to be white, the other is in blackface makeup
- Text indicating a minstrel show
- Ornate decorative elements around portraits
- Small portrait of a white man in the top left corner
- Vintage poster style

2. Potential metadata elements and refinement for inclusivity:
- Description: Vintage minstrel show poster featuring two portraits
- Transcription: All text on the poster should be transcribed
- Date: Likely late 19th or early 20th century based on style
- Location: Not specified, but minstrel shows were common in the United States
- Publication info: Poster for Wm. H. West's Big Minstrel Jubilee
- Contextual info: Minstrel shows were a form of racist entertainment
- Entire image info: Color lithograph poster
- Objects: Portraits, oval frames, decorative elements
- Actions: Advertising, performing (implied)
- People: Performers, one in black